In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from maputils import EquiValReporter
from training_utils import TrainSystem

from openmm import *

In [ ]:
dataset = dict(np.load('/storage_common/angiod/chignolin_unfolded/chignolin.data.A.kcal.npz', allow_pickle=True))
# dataset = dict(np.load('/storage_common/angiod/chignolin_unfolded/chignolin.data.A.kcal.test.npz', allow_pickle=True))
pos2unit = 1.
eng2unit = KcalPerKJ

# dataset = dict(np.load('/storage_common/angiod/chignolin_unfolded/chignolin.data.nm.kJ.test.npz', allow_pickle=True))
# pos2unit = NmPerAngstrom
# eng2unit = 1.

In [ ]:
equvalrep = EquiValReporter(dataset=dataset)

equvalrep.bondMapper(config_file_path="config/bond_config.yaml")
equvalrep.angleMapper(conf_angles_path="test_conf/config.angles.yaml")
equvalrep.improperDihedralMapper(conf_angles_path="test_conf/config.dihedrals.yaml")
equvalrep.beadChargeMapper()
equvalrep.reportEquiVals(reportPath='test_conf/')

conf_bonds:        dict = equvalrep.getBonds()
conf_angles:       dict = equvalrep.getAngles()
conf_dihedrals:    dict = equvalrep.getImproperDihs()
conf_bead_charges: dict = equvalrep.getBeadCharges()

In [ ]:
model_weights = 'models/A.kcal.best.pth'
model_weights = None

system = TrainSystem(
    dataset,
    conf_bonds,
    conf_angles,
    conf_dihedrals,
    conf_bead_charges,
    model_weights=model_weights,
    pos2unit=pos2unit,
    eng2unit=eng2unit,
)

model = system.initiateTraining(
    epochs=1,
    batch_size=100,
    patience=10,
    checkpoint_every=None,
    model_name='A.kcal',
    device='cuda:0',
    )

In [ ]:
for n, p in model.named_parameters():
    print(n, p.shape)

In [ ]:
# MOVE TO DEDICATED NOTEBOOK/SCRIPT #


# # Assign directory
# input_directory = 'StartingStructures'

# output_directory = '../datasets/NoPBCChiignolinVoid'

# trj_list = []

# for name in os.listdir(output_directory):
#     if name.endswith(".trr"):
#         trj_list.append(output_directory + '/' + name)

# print(trj_list)

# for sim_path in trj_list:

#     config = {
#         'mapping': 'martini3',
#         'input': 'StartingStructures/chignolin_frames0.pdb',
#         'inputtraj': [sim_path],
#         'selection': 'protein',
#         'output': 'pro.gro',
#         'outputtraj': 'xtc',
#         #'trajslice': slice(100,10000)
#     }

#     # Map atoms to beads (only protein for now)
#     mapping = Mapper(config)
#     mapping.map()
#     try:
#         dataset["bead_forces"] = np.append(dataset["bead_forces"], mapping.dataset['bead_forces'],axis=0)
#         dataset["bead_pos"] = np.append(dataset["bead_pos"], mapping.dataset['bead_pos'],axis=0)

#     except:
#         dataset = mapping.dataset

#     print(dataset['bead_forces'].shape)
#     # force_set = forces[:,162]

# current_dir = os.getcwd()

# output_file = os.path.join(current_dir, "dataset.NoPBCVoidALLFrames.npz")
# np.savez(output_file, **dataset)
# print(f"{output_file} successfully saved!")

In [ ]:
system.plotLosses(truncate=10)

In [ ]:
system.plotForceMagnitudeMatching(bead_index=0,to_frame=30)

In [ ]:
system.plotForceMathingByFrame(frame=10)

In [ ]:
system.plotInitialForceGuess()

In [ ]:
system.plotForceMathing(bead_index=0,to_frame=40)

In [ ]:
system.plotValForceMathing(bead_index=0,to_frame=14)

In [ ]:
system.plotABSForceMathing(frame=30)

In [ ]:
system.state_dict()